In [232]:
#importing the libraries
import nltk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
import pickle

## Importing the Dataset

In [8]:
#loading the data
data = pd.read_csv('20191226-reviews.csv')
items = pd.read_csv('20191226-items.csv')

In [171]:
data

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0
...,...,...,...,...,...,...,...,...
67981,B081H6STQQ,jande,5,"August 16, 2019",False,"Awesome Phone, but finger scanner is a big mis...",I love the camera on this phone. The screen is...,1.0
67982,B081H6STQQ,2cool4u,5,"September 14, 2019",False,Simply Amazing!,I've been an Xperia user for several years and...,1.0
67983,B081H6STQQ,simon,5,"July 14, 2019",False,"great phon3, but many bugs need to fix. still ...",buy one more for my cousin,NaN
67984,B081TJFVCJ,Tobiasz Jedrysiak,5,"December 24, 2019",True,Phone is like new,Product looks and works like new. Very much re...,NaN


In [10]:
items

,asin,brand,title,url,image,rating,reviewUrl,totalReviews,price,originalPrice
0,B0000SX2UC,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.00
1,B0009N5L7K,Motorola,Motorola I265 phone,https://www.amazon.com/Motorola-i265-I265-phon...,https://m.media-amazon.com/images/I/419WBAVDAR...,3.0,https://www.amazon.com/product-reviews/B0009N5L7K,7,49.95,0.00
2,B000SKTZ0S,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,2.7,https://www.amazon.com/product-reviews/B000SKTZ0S,22,99.99,0.00
3,B001AO4OUC,Motorola,Motorola i335 Cell Phone Boost Mobile,https://www.amazon.com/Motorola-i335-Phone-Boo...,https://m.media-amazon.com/images/I/710UO8gdT+...,3.3,https://www.amazon.com/product-reviews/B001AO4OUC,21,0.00,0.00
4,B001DCJAJG,Motorola,Motorola V365 no contract cellular phone AT&T,https://www.amazon.com/Motorola-V365-contract-...,https://m.media-amazon.com/images/I/61LYNCVrrK...,3.1,https://www.amazon.com/product-reviews/B001DCJAJG,12,149.99,0.00
...,...,...,...,...,...,...,...,...,...,...
715,B07ZPKZSSC,Apple,"Apple iPhone 11 Pro, 64GB, Fully Unlocked - Sp...",https://www.amazon.com/Apple-iPhone-64GB-Fully...,https://m.media-amazon.com/images/I/41wDuEW9iZ...,1.0,https://www.amazon.com/product-reviews/B07ZPKZSSC,1,949.00,0.00
716,B07ZQSGP53,Xiaomi,"Xiaomi Redmi Note 8, 32GB/3GB RAM 6.3"" FHD+ Di...",https://www.amazon.com/Xiaomi-Display-Snapdrag...,https://m.media-amazon.com/images/I/41foh4FKHE...,4.6,https://www.amazon.com/product-reviews/B07ZQSGP53,3,150.96,0.00
717,B081H6STQQ,Sony,Sony Xperia 1 Unlocked Smartphone and WH1000XM...,https://www.amazon.com/Sony-Smartphone-WH1000X...,https://m.media-amazon.com/images/I/51zZTAXZTP...,4.5,https://www.amazon.com/product-reviews/B081H6STQQ,70,948.00,0.00
718,B081TJFVCJ,Apple,"Apple iPhone X, 64GB, Gray - Fully Unlocked (R...",https://www.amazon.com/Apple-iPhone-64GB-Gray-...,https://m.media-amazon.com/images/I/71yMgOenT5...,5.0,https://www.amazon.com/product-reviews/B081TJFVCJ,1,478.97,0.00


In [12]:
# stopwords and stemming
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaibh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Removing Punctuations & Number

In [54]:
def remove_PN(data):
    review = []
    for i in range(data.shape[0]):
        review.append(re.sub('[^a-zA-Z]' , ' ' , str(data[i])))
    return review

In [55]:
review = remove_PN(data['body'])

In [56]:
len(review)

67986

In [57]:
review[0]

'I had the Samsung A    for awhile which is absolute doo doo  You can read my review on it and detect my rage at the stupid thing  It finally died on me so I used this Nokia phone I bought in a garage sale for     I wonder y she sold it so cheap     Bad       I hate the menu  It takes forever to get to what you want because you have to scroll endlessly  Usually phones have numbered categories so u can simply press the   and get where you want to go       It s a pain to put it on silent or vibrate  If you re in class and it rings  you have to turn it off immediately  There s no fast way to silence the damn thing  Always remember to put it on silent  I learned that the hard way       It s so true about the case  It s a mission to get off and will break ur nails in the process  Also  you ll damage the case each time u try  For some reason the phone started giving me problems once I did succeed in opening it       Buttons could be a bit bigger  Vibration could be stronger  Good       Recep

## Convert each word into Lowercase

In [58]:
def to_lower(data):
    review = []
    for i in range(len(data)):
        review.append(data[i].lower())
    return review

In [59]:
review = to_lower(review)

In [61]:
review[0]

'i had the samsung a    for awhile which is absolute doo doo  you can read my review on it and detect my rage at the stupid thing  it finally died on me so i used this nokia phone i bought in a garage sale for     i wonder y she sold it so cheap     bad       i hate the menu  it takes forever to get to what you want because you have to scroll endlessly  usually phones have numbered categories so u can simply press the   and get where you want to go       it s a pain to put it on silent or vibrate  if you re in class and it rings  you have to turn it off immediately  there s no fast way to silence the damn thing  always remember to put it on silent  i learned that the hard way       it s so true about the case  it s a mission to get off and will break ur nails in the process  also  you ll damage the case each time u try  for some reason the phone started giving me problems once i did succeed in opening it       buttons could be a bit bigger  vibration could be stronger  good       recep

## Apply Stemming

In [63]:
def to_Steam(data):
    review = []
    ps = PorterStemmer()
    for i in range(len(data)):
        sent = data[i].split()
        sent = [ps.stem(word) for word in sent if word not in set(stopwords.words('english'))]
        sent = ' '.join(sent)
        review.append(sent)
    return review

In [75]:
%%time
revew = to_Steam(review)

Wall time: 31min 10s


## Creating Features

In [317]:
cv = CountVectorizer(max_features = 2000 , decode_error = False)

In [318]:
count = cv.fit_transform(review)
X = count.toarray()

In [319]:
y = data['rating']
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [320]:
y = keras.utils.np_utils.to_categorical(y)

In [323]:
with open('feature.pkl', 'wb') as file:
    pickle.dump(cv.vocabulary_ ,file)

## Train Test Split

In [324]:
X_train , X_test, y_train , y_test = train_test_split(X , y , test_size = 0.20 , random_state = 10)

# Model Building

## Initializing Model

In [325]:
model = Sequential()

## Adding Input Layer


In [326]:
model.add(Dense(128 , activation = 'relu' , input_dim = 2000 , name = 'input_1'))

## Adding Hidden Layer

In [327]:
model.add(Dense(32 , activation = 'relu' , name = 'dense_1'))

## Adding Output Layer

In [328]:
model.add(Dense(5 , activation = 'softmax' , name = 'output_1'))

## Configuring the learning process

In [329]:
model.compile(optimizer='adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

## Training Model

In [330]:
model.fit(X_train , y_train , epochs = 25 , batch_size = 128)

Epoch 1/25
54388/54388 [==============================] - 3s 56us/step - loss: 0.8837 - accuracy: 0.6859
Epoch 2/25
54388/54388 [==============================] - 3s 52us/step - loss: 0.7139 - accuracy: 0.7350
Epoch 3/25
54388/54388 [==============================] - 3s 51us/step - loss: 0.6352 - accuracy: 0.7628
Epoch 4/25
54388/54388 [==============================] - 3s 50us/step - loss: 0.5487 - accuracy: 0.7997
Epoch 5/25
54388/54388 [==============================] - 3s 60us/step - loss: 0.4533 - accuracy: 0.8413
Epoch 6/25
54388/54388 [==============================] - 3s 52us/step - loss: 0.3612 - accuracy: 0.8789
Epoch 7/25
54388/54388 [==============================] - 3s 49us/step - loss: 0.2874 - accuracy: 0.9083
Epoch 8/25
54388/54388 [==============================] - 3s 50us/step - loss: 0.2375 - accuracy: 0.9264
Epoch 9/25
54388/54388 [==============================] - 3s 49us/step - loss: 0.2010 - accuracy: 0.9391
Epoch 10/25
54388/54388 [==============================

## Saving Model

In [331]:
model.save('mymodel.h5')

## Prediction

In [332]:
# Laod The Model
model = load_model('mymodel.h5')

In [333]:
Labels = [1, 2, 3, 4, 5]

In [334]:
def Predict(data):
    pred = model.predict(data)
    pred = np.argmax(pred , axis = 1)
    result = [Labels[i] for i in pred]
    return result

In [335]:
result = Predict(X_test[:10])

In [336]:
result

[1, 5, 5, 1, 5, 5, 5, 5, 5, 5]

In [337]:
real = [Labels[i] for i in np.argmax(y_test[:10] , axis =1)]

In [338]:
real

[1, 5, 5, 1, 5, 5, 4, 5, 5, 5]

### New Example

In [354]:
text = ['it is very bad product']
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))

In [355]:
text = loaded_vec.fit_transform(text).toarray()

In [356]:
text.shape

(1, 2000)

In [357]:
result = Predict(text)
result

[1]